In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%matplotlib notebook

import torch
# from testing.infer import *
import os
import numpy as np
from mmcv.utils import Config, DictAction, get_git_hash
from mmseg.models import build_lanedetector
from mmengine.model import revert_sync_batchnorm
from mmseg.datasets import build_dataloader, build_dataset

np.set_printoptions(suppress=True)

import matplotlib.pyplot as plt

/opt/conda/envs/anchor3dlane/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = Config.fromfile("/home/ec2-user/vardeep_ws/Anchor3DLane/configs/test_config.py")

model = build_lanedetector(cfg.model)

model.init_weights()

model = revert_sync_batchnorm(model)

dataset = build_dataset(cfg.data.test)

loader_cfg = dict(
    # cfg.gpus will be ignored if distributed
    num_gpus=1,
    dist=False,
    # seed=cfg.seed,
    drop_last=True,
    shuffle=cfg.data_shuffle,
    persistent_workers=False)
# The overall dataloader settings
loader_cfg.update({
    k: v
    for k, v in cfg.data.items() if k not in [
        'train', 'val', 'test', 'train_dataloader', 'val_dataloader',
        'test_dataloader'
    ]
})
checkpoint = torch.load("/home/ec2-user/vardeep_ws/Anchor3DLane/work_dir/openlane.pth")
model.load_state_dict(checkpoint['state_dict'])



2024-05-02 16:09:24,167 - mmcv - INFO - initialize ResNetV1c with init_cfg [{'type': 'Kaiming', 'layer': 'Conv2d'}, {'type': 'Constant', 'val': 1, 'layer': ['_BatchNorm', 'GroupNorm']}]


anchor: 4431


2024-05-02 16:09:24,245 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-05-02 16:09:24,247 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-05-02 16:09:24,249 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-05-02 16:09:24,250 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-05-02 16:09:24,252 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-05-02 16:09:24,254 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-05-02 16:09:24,256 - mmcv - INFO - initialize BasicBlock with init_cfg {'type': 'Constant', 'val': 0, 'override': {'name': 'norm2'}}
2024-05-02 16:09:24,258 - mmcv - I

is_resample: True
Now loading annotations...
after load annotation
find 8800 samples in data/OpenLane/data_lists/training.txt.


<All keys matched successfully>

In [3]:
import pickle 
with open('data.pickle', 'rb') as handle:
    data = pickle.load(handle)
for i in range(len(data['img_metas']['ori_shape'])):
    data['img_metas']['ori_shape'][i] = data['img_metas']['ori_shape'][i].cpu()
for i in range(len(data['gt_3dlanes'])):
    data['gt_3dlanes'][i] = data['gt_3dlanes'][i].cpu()


In [4]:
test_img = data['img'].squeeze()
print(data['img'].shape)
# test_img = (test_img - test_img.min()) / (test_img.max() - test_img.min())
# plt.imshow(test_img.permute(1, 2, 0))
# plt.savefig('foo.png')

torch.Size([1, 3, 360, 480])


In [5]:
model.eval()
model_out = model(**data)
print(model_out)

{'reg_proposals': tensor([[[ 0.0000,  0.0000,  0.0000,  ..., -4.3536, -4.3624, -5.6798],
         [ 0.0000,  0.0000,  0.0000,  ..., -4.0584, -4.0440, -3.3012],
         [ 0.0000,  0.0000,  0.0000,  ..., -3.9387, -3.9319, -4.1316],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ..., -3.8156, -3.8204, -5.4381],
         [ 0.0000,  0.0000,  0.0000,  ..., -4.2803, -4.2727, -5.3367],
         [ 0.0000,  0.0000,  0.0000,  ..., -3.6509, -3.6293, -5.6673]]],
       grad_fn=<CopySlices>), 'anchors': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]]), 'proposals_list': [(tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
         -1.8159e+00, -1.7828e+00, -1.8062e+00, -1.7638e+00, -1.7371e+00,
         -1.7145e+00, -1.6817e+00, -1.6622e+00, -1.6703e+00, -1.6672

/opt/conda/envs/anchor3dlane/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/opt/conda/envs/anchor3dlane/lib/python3.9/site-packages/torch/nn/functional.py:3981: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


In [6]:
engine_filtered_out = np.load("engine_res.npy")

In [7]:
model_filtered_out = model_out['proposals_list'][0][0].detach().numpy().reshape(1, -1)

In [17]:
print(model_filtered_out[:, 25:45])

[[-0.01127755 -0.00445186  0.02093108  0.03239022  0.06046604  0.08697284
   0.09998263  0.13050224  0.14310032  0.15150017  0.17145963  0.19230302
   0.20028634  0.19716918  0.21368638  0.22949193  0.23501706  0.25053477
   0.2383475   0.26405343]]


In [9]:
Error_axis0 = (np.square(engine_filtered_out - model_filtered_out)).mean(axis=0)
Error_axis1 = (np.square(engine_filtered_out - model_filtered_out)).mean(axis=1)

In [10]:
print(Error_axis1)

[0.11046792]


In [106]:
# plotting the output
x = (model_filtered_out[:, 5:25]).squeeze()
z = (model_filtered_out[:, 25:45]).squeeze()
y = (np.linspace(1, 20, 20) * 5)

x2 = (model_filtered_out[:, 86+5:86+25]).squeeze()
z2 = (model_filtered_out[:, 86+25:86+45]).squeeze()

In [107]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')

ax.set_xticks([-0.1,-0.2, -0.5, -1, 0, 2, 4, 6])
ax.set_yticks([-0.1, -0.2, -0.3, -0.5, -0.7, 2, 4, 6])
ax.set_zticks([-0.1, -0.2, -0.5, 0, 2, 4, 6])

ax.axes.set_xlim3d(left=0.2, right=9.8) 
ax.axes.set_ylim3d(bottom=0.2, top=9.8) 
ax.axes.set_zlim3d(bottom=0.2, top=9.8) 

ax.plot(x, y , z)
ax.plot(x2, y, z2)

plt.xlabel("X")
plt.ylabel("Y")




Text(0.5, 0, 'Y')

In [108]:
plt.title('Torch Output [without vis filter]')
plt.savefig('assets/torch_without_vis_filter.png')

In [16]:
model_filtered_out[0][:86]

array([  0.        ,   0.        ,   0.        ,   1.        ,
         0.        ,  -1.8159149 ,  -1.7827749 ,  -1.8061696 ,
        -1.763845  ,  -1.7371167 ,  -1.7145038 ,  -1.6817397 ,
        -1.6621883 ,  -1.6703012 ,  -1.6671634 ,  -1.6607778 ,
        -1.6775253 ,  -1.7301773 ,  -1.7595541 ,  -1.7815778 ,
        -1.7917117 ,  -1.7778251 ,  -1.767652  ,  -1.6973653 ,
        -1.669321  ,  -0.01127756,  -0.00445189,   0.02093107,
         0.0323902 ,   0.06046601,   0.08697283,   0.09998262,
         0.13050224,   0.14310029,   0.15150017,   0.17145965,
         0.19230297,   0.2002863 ,   0.19716918,   0.21368635,
         0.2294919 ,   0.235017  ,   0.25053477,   0.23834744,
         0.2640534 ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   1.        ,   1.        ,
         1.        ,   1.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  

In [18]:
z

array([-0.01127756, -0.00445189,  0.02093107,  0.0323902 ,  0.06046601,
        0.08697283,  0.09998262,  0.13050224,  0.14310029,  0.15150017,
        0.17145965,  0.19230297,  0.2002863 ,  0.19716918,  0.21368635,
        0.2294919 ,  0.235017  ,  0.25053477,  0.23834744,  0.2640534 ],
      dtype=float32)